In [1]:
pip install cryptography


Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import messagebox
import smtplib
import sqlite3
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from cryptography.fernet import Fernet

# Encryption key (in a real application, store this securely)
key = Fernet.generate_key()
cipher_suite = Fernet(key)

class MailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Mail Application")
        self.root.geometry("400x400")

        # Database setup
        self.conn = sqlite3.connect('mail_settings.db')
        self.create_table()

        # Email and Password
        tk.Label(self.root, text="Email:").pack(pady=5)
        self.email_entry = tk.Entry(self.root, width=50)
        self.email_entry.pack(pady=5)

        tk.Label(self.root, text="Password:").pack(pady=5)
        self.password_entry = tk.Entry(self.root, width=50, show='*')
        self.password_entry.pack(pady=5)

        # SMTP Server and Port
        tk.Label(self.root, text="SMTP Server:").pack(pady=5)
        self.smtp_server_entry = tk.Entry(self.root, width=50)
        self.smtp_server_entry.pack(pady=5)

        tk.Label(self.root, text="SMTP Port:").pack(pady=5)
        self.smtp_port_entry = tk.Entry(self.root, width=50)
        self.smtp_port_entry.pack(pady=5)

        # Buttons
        self.save_button = tk.Button(self.root, text="Save Settings", command=self.save_settings)
        self.save_button.pack(pady=20)

        self.send_button = tk.Button(self.root, text="Send Email", command=self.send_email)
        self.send_button.pack(pady=20)

        # Load settings if available
        self.load_settings()

    def create_table(self):
        cursor = self.conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS settings
                          (email TEXT, password TEXT, smtp_server TEXT, smtp_port TEXT)''')
        self.conn.commit()

    def save_settings(self):
        email = self.email_entry.get()
        password = self.password_entry.get()
        smtp_server = self.smtp_server_entry.get()
        smtp_port = self.smtp_port_entry.get()

        encrypted_password = cipher_suite.encrypt(password.encode()).decode()

        cursor = self.conn.cursor()
        cursor.execute("DELETE FROM settings")
        cursor.execute("INSERT INTO settings VALUES (?, ?, ?, ?)",
                       (email, encrypted_password, smtp_server, smtp_port))
        self.conn.commit()
        messagebox.showinfo("Info", "Settings saved successfully.")

    def load_settings(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM settings")
        row = cursor.fetchone()
        if row:
            email, encrypted_password, smtp_server, smtp_port = row
            self.email_entry.insert(0, email)
            self.password_entry.insert(0, cipher_suite.decrypt(encrypted_password.encode()).decode())
            self.smtp_server_entry.insert(0, smtp_server)
            self.smtp_port_entry.insert(0, smtp_port)

    def send_email(self):
        email = self.email_entry.get()
        password = self.password_entry.get()
        smtp_server = self.smtp_server_entry.get()
        smtp_port = self.smtp_port_entry.get()

        # Create the email content
        msg = MIMEMultipart()
        msg['From'] = email
        msg['To'] = email
        msg['Subject'] = "Test Email"
        body = "This is a test email from the mail application."
        msg.attach(MIMEText(body, 'plain'))

        try:
            server = smtplib.SMTP(smtp_server, int(smtp_port))
            server.starttls()
            server.login(email, password)
            text = msg.as_string()
            server.sendmail(email, email, text)
            server.quit()
            messagebox.showinfo("Info", "Email sent successfully.")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to send email. Error: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = MailApp(root)
    root.mainloop()
